In [14]:
from keras.models import model_from_json
from keras.optimizers import SGD

json_file = open('Binary_Code.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("saved_best_model.hdf5")
#loaded_model.load_weights("Binary_Code_model_weights.hdf5")
print("Loaded model from disk")


# Evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model from disk


In [15]:
import cv2
import numpy as np
import glob, os
import scipy.ndimage

In [17]:
# Instead of cv2.imread use ndimage.imread
from scipy import ndimage,misc
m_test = []
m_pred = []

y_tests_cf = []
y_preds_cf = []

malignantImages = glob.glob("C:\\Users\\Aiden\\Documents\\My Documents\\Study Material\\Internship\\CancerSegmentation\\SkinCancer\\ImagesMalignant\\ISIC_*.jpg")


#once the model is trained to test a new image
for malignantImage in malignantImages:
    categories=['Benignant','Malignant']
    images_to_predict = ndimage.imread(malignantImage)
    images_to_predict= misc.imresize(images_to_predict, (112,150))
    images_to_predict=np.array(images_to_predict).reshape(-1,112,150,3)/255
    print(categories[loaded_model.predict_classes(images_to_predict)[0][0]])
    m_test.append(1)
    y_tests_cf.append(1)
    m_pred.append(loaded_model.predict_classes(images_to_predict)[0][0])
    y_preds_cf.append(loaded_model.predict_classes(images_to_predict)[0][0])

C:\Users\Aiden\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  from ipykernel import kernelapp as app
C:\Users\Aiden\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  app.launch_new_instance()


Malignant
Malignant
Malignant
Malignant
Malignant
Malignant


In [18]:
len(malignantImages)

6

In [19]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(m_test, m_pred).ravel()

print("True Negatives: %s" % tn)
print("False Positives: %s" % fp)
print("False Negatives: %s" % fn)
print("True Positives: %s" % tp)

ValueError: not enough values to unpack (expected 4, got 1)

In [20]:
b_test = []
b_pred = []

benignantImages = glob.glob("C:\\Users\\Aiden\\Documents\\My Documents\\Study Material\\Internship\\CancerSegmentation\\SkinCancer\\ImagesBenign\\ISIC_*.jpg")

#once the model is trained to test a new image
for benignImage in benignantImages:
    categories=['Benignant','Malignant']
    images_to_predict = ndimage.imread(benignImage)
    images_to_predict= misc.imresize(images_to_predict, (112,150))
    images_to_predict=np.array(images_to_predict).reshape(-1,112,150,3)/255
    #print("File: ", os.path.basename(benignImage), ", Category: ", categories[loaded_model.predict_classes(images_to_predict)[0][0]], ", Probability: ", loaded_model.predict_proba(images_to_predict)[0][0])
    print(categories[loaded_model.predict_classes(images_to_predict)[0][0]])
    b_test.append(0)
    y_tests_cf.append(0)
    b_pred.append(loaded_model.predict_classes(images_to_predict)[0][0])  
    y_preds_cf.append(loaded_model.predict_classes(images_to_predict)[0][0])  

C:\Users\Aiden\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  if __name__ == '__main__':
C:\Users\Aiden\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # Remove the CWD from sys.path while we load stuff.


Malignant
Malignant
Malignant
Malignant
Malignant
Malignant


In [ ]:
len(benignantImages)

In [21]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(b_test, b_pred).ravel()

print("True Negatives: %s" % tn)
print("False Positives: %s" % fp)
print("False Negatives: %s" % fn)
print("True Positives: %s" % tp)

True Negatives: 0
False Positives: 6
False Negatives: 0
True Positives: 0


# Overall Confusion Matrix

In [22]:
tn, fp, fn, tp = confusion_matrix(y_tests_cf, y_preds_cf).ravel()

print("True Negatives: %s" % tn)
print("False Positives: %s" % fp)
print("False Negatives: %s" % fn)
print("True Positives: %s" % tp)

True Negatives: 0
False Positives: 6
False Negatives: 0
True Positives: 6


In [23]:
#Takes name of image returns category and probability of the category
def pred(Image):
    categories = ['Benignant','Malignant']
    image=cv2.imread(Image)
    image=cv2.resize(image, (112,150))
    img = np.array(image).reshape(-1,112,150,3)/255
    #pred = loaded_model.predict(img)[0]
    #top = np.argsort(proba)[-1]
    #category = categories[top]
    #proba = pred[top]
    category=categories[loaded_model.predict_classes(img)[0][0]]
    proba=loaded_model.predict_proba(img)[0][0]
    return [category,proba]

def compare(Image1,Image2):
    img_pred1=pred(Image1)
    print((img_pred1))
    img_pred2=pred(Image2)
    print((img_pred2))
    if(img_pred1[0]=='Malignant'):
        if(img_pred1[1] > img_pred2[1]):
            #Improve
            return ((img_pred1[1]-img_pred2[1]), 0, 0)
        elif(img_pred1[1] ==img_pred2[1]):
            #Same
            return (0,1,0)
        elif(img_pred1[1] < img_pred2[1]):
            #Worse
            return (0, 0, (img_pred2[1]-img_pred1[1]))
    if(img_pred1[0]=='Benignant'):
        if(img_pred1[1] < img_pred2[1]):
            #Improve
            return ((img_pred1[1]-img_pred2[1]), 0, 0)
        elif(img_pred1[1] ==img_pred2[1]):
            #Same
            return (0,1,0)
        elif(img_pred1[1] > img_pred2[1]):
            #Worse
            return (0, 0, (img_pred2[1]-img_pred1[1]))

In [28]:
#copy these two files in folder where this notebook is saved  
op=compare("ISIC_00000.jpg","ISIC_00002.jpg")
print(op)

['Malignant', 0.97345126]
['Malignant', 0.6408044]
(0.33264685, 0, 0)
